Not long ago, I needed to parse some HTML tables from our confluence website at work. I first thought: I'm gonna need [requests](http://docs.python-requests.org/en/master/) and [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/). As HTML tables are well defined, I did some quick googling to see if there was some recipe or lib to parse them and I found a link to [pandas](https://pandas.pydata.org). What? Can pandas do that too?

I have been using pandas for quite some time and have used read_csv, read_excel, even read_sql, but I had missed read_html!

# Reading excel file with pandas

Before to look at HTML tables, I want to show a quick example on how to read an excel file with pandas. The API is really nice. If I have to look at some excel data, I go directly to pandas.

So let's download a sample file file:

In [1]:
import io
import requests
import pandas as pd
from zipfile import ZipFile

In [2]:
r = requests.get('http://www.contextures.com/SampleData.zip')
ZipFile(io.BytesIO(r.content)).extractall()

This created the *SampleData.xlsx* file that includes four sheets: Instructions, SalesOrders, SampleNumbers and MyLinks. Only the *SalesOrders* sheet includes tabular data:
![SampleData](/images/read_html/sample_data_xlsx.png)
So let's read it.

In [3]:
df = pd.read_excel('SampleData.xlsx', sheet_name='SalesOrders')

In [4]:
df.head()

,OrderDate,Region,Rep,Item,Units,Unit Cost,Total
0,2016-01-06,East,Jones,Pencil,95,1.99,189.05
1,2016-01-23,Central,Kivell,Binder,50,19.99,999.50
2,2016-02-09,Central,Jardine,Pencil,36,4.99,179.64
3,2016-02-26,Central,Gill,Pen,27,19.99,539.73
4,2016-03-15,West,Sorvino,Pencil,56,2.99,167.44


That's it. One line and you have your data in a DataFrame that you can easily manipulate, filter, convert and display in a jupyter notebook. Can it be easier than that?

# Parsing HTML Tables

So let's go back to HTML tables and look at [pandas.read_html](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_html.html).

The function accepts:
> A URL, a file-like object, or a raw string containing HTML.

Let's start with a basic HTML table in a raw string.

## Parsing raw string

In [5]:
html_string = """
<table>
  <thead>
    <tr>
      <th>Programming Language</th>
      <th>Creator</th> 
      <th>Year</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>C</td>
      <td>Dennis Ritchie</td> 
      <td>1972</td>
    </tr>
    <tr>
      <td>Python</td>
      <td>Guido Van Rossum</td> 
      <td>1989</td>
    </tr>
    <tr>
      <td>Ruby</td>
      <td>Yukihiro Matsumoto</td> 
      <td>1995</td>
    </tr>
  </tbody>
</table>
"""

We can render the table using IPython [display_html](http://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html#IPython.display.display_html) function:

In [6]:
from IPython.display import display_html
display_html(html_string, raw=True)

Programming Language,Creator,Year
C,Dennis Ritchie,1972
Python,Guido Van Rossum,1989
Ruby,Yukihiro Matsumoto,1995


Let's import this HTML table in a DataFrame. Note that the function `read_html` always returns a list of DataFrame objects:

In [7]:
dfs = pd.read_html(html_string)
dfs

[  Programming Language             Creator  Year
 0                    C      Dennis Ritchie  1972
 1               Python    Guido Van Rossum  1989
 2                 Ruby  Yukihiro Matsumoto  1995]

In [8]:
df = dfs[0]
df

,Programming Language,Creator,Year
0,C,Dennis Ritchie,1972
1,Python,Guido Van Rossum,1989
2,Ruby,Yukihiro Matsumoto,1995


This looks quite similar to the raw string we rendered above, but we are printing a pandas DataFrame object here! We can apply any operation we want.

In [9]:
df[df.Year > 1975]

,Programming Language,Creator,Year
1,Python,Guido Van Rossum,1989
2,Ruby,Yukihiro Matsumoto,1995


Pandas automatically found the header to use thanks to the `<thead>` tag. It is not mandatory to define a table and is actually often missing on the web. So what happens if it's not present?

In [10]:
html_string = """
<table>
  <tr>
    <th>Programming Language</th>
    <th>Creator</th> 
    <th>Year</th>
  </tr>
  <tr>
    <td>C</td>
    <td>Dennis Ritchie</td> 
    <td>1972</td>
  </tr>
  <tr>
    <td>Python</td>
    <td>Guido Van Rossum</td> 
    <td>1989</td>
  </tr>
  <tr>
    <td>Ruby</td>
    <td>Yukihiro Matsumoto</td> 
    <td>1995</td>
  </tr>
</table>
"""

In [11]:
pd.read_html(html_string)[0]

,0,1,2
0,Programming Language,Creator,Year
1,C,Dennis Ritchie,1972
2,Python,Guido Van Rossum,1989
3,Ruby,Yukihiro Matsumoto,1995


In this case, we need to pass the row number to use as header.

In [12]:
pd.read_html(html_string, header=0)[0]

,Programming Language,Creator,Year
0,C,Dennis Ritchie,1972
1,Python,Guido Van Rossum,1989
2,Ruby,Yukihiro Matsumoto,1995


## Parsing a http URL

The same data we read in our excel file is available in a table at the following address: http://www.contextures.com/xlSampleData01.html

Let's pass this url to `read_html`:

In [13]:
dfs = pd.read_html('http://www.contextures.com/xlSampleData01.html')

In [14]:
dfs

[             0        1         2        3      4         5        6
 0    OrderDate   Region       Rep     Item  Units  UnitCost    Total
 1     1/6/2016     East     Jones   Pencil     95      1.99   189.05
 2    1/23/2016  Central    Kivell   Binder     50     19.99   999.50
 3     2/9/2016  Central   Jardine   Pencil     36      4.99   179.64
 4    2/26/2016  Central      Gill      Pen     27     19.99   539.73
 5    3/15/2016     West   Sorvino   Pencil     56      2.99   167.44
 6     4/1/2016     East     Jones   Binder     60      4.99   299.40
 7    4/18/2016  Central   Andrews   Pencil     75      1.99   149.25
 8     5/5/2016  Central   Jardine   Pencil     90      4.99   449.10
 9    5/22/2016     West  Thompson   Pencil     32      1.99    63.68
 10    6/8/2016     East     Jones   Binder     60      8.99   539.40
 11   6/25/2016  Central    Morgan   Pencil     90      4.99   449.10
 12   7/12/2016     East    Howard   Binder     29      1.99    57.71
 13   7/29/2016     

We have one table and can see that we need to pass the row number to use as header (because `<thead>` is not present).

In [15]:
dfs = pd.read_html('http://www.contextures.com/xlSampleData01.html', header=0)
dfs[0].head()

,OrderDate,Region,Rep,Item,Units,UnitCost,Total
0,1/6/2016,East,Jones,Pencil,95,1.99,189.05
1,1/23/2016,Central,Kivell,Binder,50,19.99,999.50
2,2/9/2016,Central,Jardine,Pencil,36,4.99,179.64
3,2/26/2016,Central,Gill,Pen,27,19.99,539.73
4,3/15/2016,West,Sorvino,Pencil,56,2.99,167.44


Nice!

## Parsing a https URL

The documentation states that:

> Note that lxml only accepts the http, ftp and file url protocols. If you have a URL that starts with 'https' you might try removing the 's'.

This is true, but *bs4 + html5lib* are used as a fallback when *lxml* fails. I guess this is why passing a `https` url does work. We can confirm that with a wikipedia page.


In [16]:
pd.read_html('https://en.wikipedia.org/wiki/Python_(programming_language)', header=0)[1]

,Type,mutable,Description,Syntax example
0,bool,immutable,Boolean value,True False
1,bytearray,mutable,Sequence of bytes,"bytearray(b'Some ASCII') bytearray(b""Some ASCI..."
2,bytes,immutable,Sequence of bytes,"b'Some ASCII' b""Some ASCII"" bytes([119, 105, 1..."
3,complex,immutable,Complex number with real and imaginary parts,3+2.7j
4,dict,mutable,Associative array (or dictionary) of key and v...,"{'key1': 1.0, 3: False}"
5,ellipsis,NaN,An ellipsis placeholder to be used as an index...,...
6,float,immutable,"Floating point number, system-defined precision",3.1415927
7,frozenset,immutable,"Unordered set, contains no duplicates; can con...","frozenset([4.0, 'string', True])"
8,int,immutable,Integer of unlimited magnitude[76],42
9,list,mutable,"List, can contain mixed types","[4.0, 'string', True]"


But what if the url requires authentiation?

In that case we can use [requests](http://docs.python-requests.org/en/master/) to get the HTML and pass the string to pandas!

To demonstrate authentication, we can use http://httpbin.org

We can first confirm that passing a url that requires authentication raises a 401

In [17]:
pd.read_html('https://httpbin.org/basic-auth/myuser/mypasswd')

HTTPError: HTTP Error 401: UNAUTHORIZED

In [ ]:
r = requests.get('https://httpbin.org/basic-auth/myuser/mypasswd')
r.status_code

Yes, as expected. Let's pass the username and password with requests.

In [ ]:
r = requests.get('https://httpbin.org/basic-auth/myuser/mypasswd', auth=('myuser', 'mypasswd'))
r.status_code

We could now pass `r.text` to pandas. http://httpbin.org was used to demonstrate authentication but it only returns JSON-encoded responses and no HTML. It's a testing service. So it doesn't make sense here.

The following example shows how to combine requests and pandas.

In [18]:
r = requests.get('https://en.wikipedia.org/wiki/Python_(programming_language)')
pd.read_html(r.text, header=0)[1]

,Type,mutable,Description,Syntax example
0,bool,immutable,Boolean value,True False
1,bytearray,mutable,Sequence of bytes,"bytearray(b'Some ASCII') bytearray(b""Some ASCI..."
2,bytes,immutable,Sequence of bytes,"b'Some ASCII' b""Some ASCII"" bytes([119, 105, 1..."
3,complex,immutable,Complex number with real and imaginary parts,3+2.7j
4,dict,mutable,Associative array (or dictionary) of key and v...,"{'key1': 1.0, 3: False}"
5,ellipsis,NaN,An ellipsis placeholder to be used as an index...,...
6,float,immutable,"Floating point number, system-defined precision",3.1415927
7,frozenset,immutable,"Unordered set, contains no duplicates; can con...","frozenset([4.0, 'string', True])"
8,int,immutable,Integer of unlimited magnitude[76],42
9,list,mutable,"List, can contain mixed types","[4.0, 'string', True]"


## A more complex example

We looked at some quite simple examples so far. So let's try a page with several tables: https://en.wikipedia.org/wiki/Timeline_of_programming_languages

In [19]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/Timeline_of_programming_languages')

In [20]:
len(dfs)

13

If we look at the page we have 8 tables (one per decade). Looking at our `dfs` list, we can see that the first interesting table is the fifth one and that we need to pass the row to use as header.

In [21]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/Timeline_of_programming_languages', header=0)
dfs[4]

,Year,Name,"Chief developer, company",Predecessor(s)
0,1943–45,Plankalkül (concept),Konrad Zuse,none (unique language)
1,1943–46,ENIAC coding system,"John von Neumann, John Mauchly, J. Presper Eck...",none (unique language)
2,1946,ENIAC Short Code,"Richard Clippinger, John von Neumann after Ala...",ENIAC coding system
3,1946,Von Neumann and Goldstine graphing system (Not...,John von Neumann and Herman Goldstine,ENIAC coding system
4,1947,ARC Assembly,Kathleen Booth[1][2],ENIAC coding system
5,1948,CPC Coding scheme,Howard H. Aiken,Analytical Engine order code
6,1948,Curry notation system,Haskell Curry,ENIAC coding system
7,1948,Plankalkül (concept published),Konrad Zuse,none (unique language)
8,1949,Short Code,John Mauchly and William F. Schmitt,ENIAC Short Code
9,Year,Name,"Chief developer, company",Predecessor(s)


Notice that the header was repeated in the last row (to make the table easier to read on the HTML page). We can filter that after concatenating together the 8 tables to get one DataFrame.

In [22]:
df = pd.concat(dfs[4:12])
df

,Year,Name,"Chief developer, company",Predecessor(s)
0,1943–45,Plankalkül (concept),Konrad Zuse,none (unique language)
1,1943–46,ENIAC coding system,"John von Neumann, John Mauchly, J. Presper Eck...",none (unique language)
2,1946,ENIAC Short Code,"Richard Clippinger, John von Neumann after Ala...",ENIAC coding system
3,1946,Von Neumann and Goldstine graphing system (Not...,John von Neumann and Herman Goldstine,ENIAC coding system
4,1947,ARC Assembly,Kathleen Booth[1][2],ENIAC coding system
5,1948,CPC Coding scheme,Howard H. Aiken,Analytical Engine order code
6,1948,Curry notation system,Haskell Curry,ENIAC coding system
7,1948,Plankalkül (concept published),Konrad Zuse,none (unique language)
8,1949,Short Code,John Mauchly and William F. Schmitt,ENIAC Short Code
9,Year,Name,"Chief developer, company",Predecessor(s)


Remove the extra *header* rows.

In [23]:
prog_lang = df[df.Year != 'Year']
prog_lang

,Year,Name,"Chief developer, company",Predecessor(s)
0,1943–45,Plankalkül (concept),Konrad Zuse,none (unique language)
1,1943–46,ENIAC coding system,"John von Neumann, John Mauchly, J. Presper Eck...",none (unique language)
2,1946,ENIAC Short Code,"Richard Clippinger, John von Neumann after Ala...",ENIAC coding system
3,1946,Von Neumann and Goldstine graphing system (Not...,John von Neumann and Herman Goldstine,ENIAC coding system
4,1947,ARC Assembly,Kathleen Booth[1][2],ENIAC coding system
5,1948,CPC Coding scheme,Howard H. Aiken,Analytical Engine order code
6,1948,Curry notation system,Haskell Curry,ENIAC coding system
7,1948,Plankalkül (concept published),Konrad Zuse,none (unique language)
8,1949,Short Code,John Mauchly and William F. Schmitt,ENIAC Short Code
0,1950,Short Code,"William F Schmidt, Albert B. Tonik,[3] J.R. Logan",Brief Code


In what year was Python created?

In [24]:
prog_lang[prog_lang.Name == 'Python']

,Year,Name,"Chief developer, company",Predecessor(s)
9,1991,Python,Guido van Rossum,"ABC, ALGOL 68, Icon, Modula-3"


# Conclusion

The last example should say it all.

In [25]:
import pandas as pd

dfs = pd.read_html('https://en.wikipedia.org/wiki/Timeline_of_programming_languages', header=0)
df = pd.concat(dfs[4:12])
prog_lang = df[df.Year != 'Year']

Four lines of code (including the `import`) and we have one DataFrame containing the data from 8 different HTML tables on one wikipedia page!

Do I need to say why I love Python and pandas? :-)

This post was written in a jupyter notebook.
You can find the notebook on [GitHub](https://github.com/beenje/blog/blob/master/posts/parsing-html-tables-in-python-with-pandas.ipynb) and download the conda [environment.yml](environment.yml) file to get all the dependencies I used.